# Penguins in Production - Session 6

This notebook aims to create a [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) to build an end-to-end Machine Learning system to solve the problem of classifying penguin species.

This example uses the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data).

Amazon SageMaker is free to try. Your free tier starts from the first month you create your first SageMaker resource and lasts two months. Check out the  [Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/) for more information. Also, we'll be working extensively with [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) and the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/). Keep their documentation handy.

This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

Let's ensure we are running the latest version of the SakeMaker SDK. **Restart the Kernel** after you run the following cell.

In [2]:
%load_ext autoreload
%autoreload 2

import sys
CODE_FOLDER = "code"
sys.path.append(f"./{CODE_FOLDER}")

# Session 6 - Model Monitoring

This session aims to set up a monitoring process to analyze the quality of the model predictions. For this, we need to generate ground truth for the data captured by the endpoint and compare it with a baseline performance.

Check [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html) for a brief explanation of how to use SageMaker's Model Monitoring functionality. [Monitor models for data and model quality, bias, and explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) is a much more extensive guide to Model Monitoring in Amazon SageMaker.

Here is what the Pipeline will look like at the end of this session:

<img src='penguins/images/session6-pipeline.png' alt='Session 6 Pipeline' width="600">


In [3]:
%%writefile {CODE_FOLDER}/session6.py

from session1 import *
from session2 import *
from session3 import *
from session4 import *
from session5 import *

from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

from sagemaker.inputs import CreateModelInput, TransformInput
from sagemaker.transformer import Transformer
from sagemaker.workflow.steps import CreateModelStep, TransformStep
from sagemaker.model_monitor import ModelQualityMonitor

Overwriting code/session6.py


## Step 1 - Creating Test Predictions

To create a baseline to compare the model performance, we must create predictions for the test set and compare them with the predictions from the model. We can do this by running a Batch Transform Job to predict every sample from the test dataset. We can use a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform) as part of the pipeline to run this job. You can check [Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) for more information about Batch Transform Jobs.

The Transform Step requires a model to generate predictions, so we need a Model Step that creates a model.

We also need to configure the [Batch Transform Job](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) using a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform). This Batch Transform Job will run every sample from the training dataset through the model so we can compute the baseline metrics. We can use an instance of the [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html) class to configure the job.

In [4]:
%%writefile -a {CODE_FOLDER}/session6.py

create_model_step = ModelStep(
    name="create-model",
    step_args=model.create(instance_type="ml.m5.large"),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    
    instance_type="ml.c5.xlarge",
    instance_count=1,
    
    accept="application/json",
    strategy="SingleRecord",
    assemble_with="Line",
    
    output_path=f"{S3_FILEPATH}/transform",
)

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    transformer=transformer,
    inputs=TransformInput(
        
        # We will use the test dataset we generated during the preprocessing 
        # step to run it through the model and generate predictions.
        data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs["test-baseline"].S3Output.S3Uri,

        join_source="Input",
        content_type="application/json",
        split_type="Line",
    ),
    cache_config=cache_config
)

Appending to code/session6.py


## Step 2 - Generating a Baseline

Let's now configure the [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) and feed it the data we generated in the Transform Step.

In [5]:
%%writefile -a {CODE_FOLDER}/session6.py

model_quality_location = f"{S3_FILEPATH}/monitoring/model-quality"

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",
    
    check_job_config = CheckJobConfig(
        instance_type="ml.t3.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=sagemaker_session,
        role=role,
    ),
    
    quality_check_config = ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,

        dataset_format=DatasetFormat.json(lines=True),

        # We need to specify the problem type and the fields where the prediction
        # and groundtruth are so the process knows how to interpret the results.
        problem_type="MulticlassClassification",
        inference_attribute="$.SageMakerOutput.species",
        ground_truth_attribute="species",

        output_s3_uri=model_quality_location,
    ),
    
    skip_check=True,
    register_new_baseline=True,
    model_package_group_name=model_package_group_name,
    cache_config=cache_config
)

Appending to code/session6.py


## Step 3 - Setting up Model Metrics

We can configure a new set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) using the results of the Data and Model Quality Steps.

In [6]:
%%writefile -a {CODE_FOLDER}/session6.py

model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

Appending to code/session6.py


## Step 4 - Registering the Model

We need to redefine the Model Step to register the [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) so it takes into account the new metrics.

In [7]:
%%writefile -a {CODE_FOLDER}/session6.py

register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        drift_check_baselines=drift_check_baselines,
        approval_status="Approved",

        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
    )
)

Appending to code/session6.py


## Step 5 - Setting up the Condition Step

We only want to compute the model quality baseline if the model's performance is above the predefined threshold. The [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) will gate all necessary steps to compute the baseline. 

In [8]:
%%writefile -a {CODE_FOLDER}/session6.py

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[
        create_model_step, 
        generate_test_predictions_step, 
        model_quality_baseline_step, 
        register_model_step,
        deploy_step
    ],
    else_steps=[fail_step], 
)

Appending to code/session6.py


## Step 6 - Running the Pipeline

We can now run the pipeline.

In [9]:
from session5 import *
from sagemaker.workflow.pipeline import Pipeline


pipeline = Pipeline(
    name="penguins-session6-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        train_dataset_baseline_destination,
        test_dataset_baseline_destination,
        data_capture_percentage,
        data_capture_destination,
        accuracy_threshold,
    ],
    steps=[
        preprocess_data_step, 
        data_quality_baseline_step,
        train_model_step, 
        evaluate_model_step,
        condition_step
    ],
    pipeline_definition_config=pipeline_definition_config
)

Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist or update it if it does.

In [61]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource
Using provided s3_resource


## Step 7 - Generating Ground Truth Data

To monitor our model, we need to generate ground truth data for the samples captured by the endpoint. We can simulate this by generating a random ground truth for every sample. Check [Ingest Ground Truth Labels and Merge Them With Predictions](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-merge.html) for more information about this.

In [10]:
%%writefile -a {CODE_FOLDER}/session6.py

import pandas as pd
from threading import Thread, Event
from time import sleep


ground_truth_path = f"{S3_FILEPATH}/monitoring/groundtruth" 

def generate_ground_truth_data(predictor, ground_truth_path):
    
    def _generate_ground_truth_record(inference_id):
        random.seed(inference_id)

        return {
            "groundTruthData": {
                "data": random.choice(["Adelie", "Chinstrap", "Gentoo"]),
                "encoding": "CSV",
            },
            "eventMetadata": {
                "eventId": str(inference_id),
            },
            "eventVersion": "0",
        }


    def _upload_ground_truth(records, upload_time):
        records = [json.dumps(r) for r in records]
        data = "\n".join(records)
        uri = f"{ground_truth_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"

        print(f"Uploading ground truth data to {uri}...")

        S3Uploader.upload_string_as_file_body(data, uri)    

                
    def _generate_ground_truth_data(max_records, stop_ground_truth_thread):
        while True:
            records = [_generate_ground_truth_record(i) for i in range(max_records)]
            _upload_ground_truth(records, datetime.utcnow())

            if stop_ground_truth_thread.is_set():
                break

            sleep(30)

                
    stop_ground_truth_thread = Event()
    data = pd.read_csv(LOCAL_FILEPATH).dropna()
    
    groundtruth_thread = Thread(
        target=_generate_ground_truth_data,
        args=(len(data), stop_ground_truth_thread,)
    )
    
    groundtruth_thread.start()
    
    return stop_ground_truth_thread, traffic_thread


Appending to code/session6.py


Let's start generating traffic to the endpoint and create random ground truth data.

In [65]:
waiter = sagemaker_client.get_waiter("endpoint_in_service")
waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={
        "Delay": 10,
        "MaxAttempts": 30
    }
)

predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

In [ ]:
stop_traffic_thread, traffic_thread = generate_traffic(predictor)
stop_ground_truth_thread, groundtruth_thread = generate_ground_truth_data(predictor, ground_truth_path)

## Step 8 - Scheduling the Monitoring Job

Let's set up a schedule to continuously monitor the quality of the model and compare it to the baseline we generated before. This monitoring job will use the baseline constraints we generated during the Model Quality Check Step. Check [Schedule Model Quality Monitoring Jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-schedule.html) for more information.

To set up a Model Quality Monitoring Job, we can use the [ModelQualityMonitor](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.ModelQualityMonitor) class. The [EndpointInput](https://sagemaker.readthedocs.io/en/v2.24.2/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.EndpointInput) instance configures the attribute the monitoring job should use to determine the prediction from the model.

Check [Amazon SageMaker Model Quality Monitor](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/model_quality/model_quality_churn_sdk.html) for a complete tutorial on how to run a Model Monitoring Job in SageMaker.

In [130]:
model_monitor = ModelQualityMonitor(
    instance_type="ml.m5.xlarge",
    instance_count=1,
    max_runtime_in_seconds=1800,
    role=role
)

model_monitor.create_monitoring_schedule(
    monitor_schedule_name="penguins-model-monitoring-schedule",
    
    endpoint_input = EndpointInput(
        endpoint_name=predictor.endpoint_name,

        # The endpoint returns an attribute `species` with the
        # prediction from the model. That's the attribute we want to
        # use to compare with the groundtruth.
        inference_attribute="species",

        destination="/opt/ml/processing/input_data",
    ),
    
    problem_type="MulticlassClassification",
    ground_truth_input=ground_truth_path,
    
    constraints=f"{model_quality_location}/constraints.json",
    
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    output_s3_uri=f"{S3_FILEPATH}/monitoring/model-quality",
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: penguins-model-monitoring-schedule


## Step 9 - Checking Monitoring Violations

We can check the results of the monitoring job by looking at whether it generated any violations.

In [138]:
description = model_monitor.describe_schedule()
description

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:325223348818:monitoring-schedule/penguins-model-monitoring-schedule',
 'MonitoringScheduleName': 'penguins-model-monitoring-schedule',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2023, 6, 28, 12, 40, 7, 350000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 6, 28, 13, 32, 55, 997000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2023-06-28-12-40-06-773',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'penguins-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'penguins-model-monitoring-schedule',
  'ScheduledTime': datetime.datetime(2023, 6, 28, 13, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2023, 6, 28, 13, 8, 8, 325000, tzinfo=tzlocal()),
  'LastModifiedTime': datetime.dateti

In [139]:
status = description["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"]
print(f"Status: {status}")

if status == "CompletedWithViolations":
    processing_job_arn = description["LastMonitoringExecutionSummary"]["ProcessingJobArn"]
    execution = MonitoringExecution.from_processing_arn(sagemaker_session=sagemaker_session, processing_job_arn=processing_job_arn)
    execution_destination = execution.output.destination
    
    violations_filepath = os.path.join(execution_destination, "constraint_violations.json")
    violations = json.loads(S3Downloader.read_file(violations_filepath))["violations"]
    
    print(json.dumps(violations, indent=2))

Status: CompletedWithViolations
[
  {
    "constraint_check_type": "LessThanThreshold",
    "description": "Metric weightedF2 with 0.35210111607474637 +/- 2.489284148228981E-5 was LessThanThreshold '0.9807900477958932'",
    "metric_name": "weightedF2"
  },
  {
    "constraint_check_type": "LessThanThreshold",
    "description": "Metric accuracy with 0.35944503735325506 +/- 1.8368026615194536E-5 was LessThanThreshold '0.9807692307692307'",
    "metric_name": "accuracy"
  },
  {
    "constraint_check_type": "LessThanThreshold",
    "description": "Metric weightedRecall with 0.359445037353255 +/- 1.8368026615206323E-5 was LessThanThreshold '0.9807692307692308'",
    "metric_name": "weightedRecall"
  },
  {
    "constraint_check_type": "LessThanThreshold",
    "description": "Metric weightedPrecision with 0.35481246824881185 +/- 3.635184951719884E-5 was LessThanThreshold '0.9835164835164835'",
    "metric_name": "weightedPrecision"
  },
  {
    "constraint_check_type": "LessThanThreshold"

## Step 10 - Cleaning up

Let's stop the monitoring job by deleting the monitoring schedule we created before.

In [62]:
delete_monitoring_schedule(model_monitor)

NameError: name 'model_monitor' is not defined

We also need to stop the threads generating predictions and ground truth data.

In [63]:
stop_traffic_thread.set()
stop_ground_truth_thread.set()

traffic_thread.join()
groundtruth_thread.join()

NameError: name 'stop_traffic_thread' is not defined

Finally, let's delete the endpoint.

In [66]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: penguins-endpoint-config-0630082428
INFO:sagemaker:Deleting endpoint with name: penguins-endpoint
